# NYC Street Trees Mapmaking

### The following is meant more as a sandbox than a polished workbook, for making various maps. Be aware that the html maps get quite large. Enjoy :)

## Cleaning and exploring the data
* Import the necessary libraries.

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('ticks')
import folium
import geopandas as gpd

import pickle

from functions import *
%load_ext autoreload
%autoreload 2

from scipy import stats
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import BallTree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks


import warnings
warnings.filterwarnings('ignore')

# POSSIBLY
from collections import Counter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


* Import the data and take a look.

In [3]:
trees_initial = pd.read_csv('data/2015StreetTreesCensus_TREES.csv')
trees_initial.head()

,created_at,tree_id,block_id,the_geom,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,...,st_assem,st_senate,nta,nta_name,boro_ct,state,Latitude,longitude,x_sp,y_sp
0,08/27/2015,180683,348711,POINT (-73.84421521958048 40.723091773924274),3,0,OnCurb,Alive,Fair,Acer rubrum,...,28,16,QN17,Forest Hills,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.768749
1,09/03/2015,200540,315986,POINT (-73.81867945834878 40.79411066708779),21,0,OnCurb,Alive,Fair,Quercus palustris,...,27,11,QN49,Whitestone,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.837379
2,09/05/2015,204026,218365,POINT (-73.93660770459083 40.717580740099116),3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,50,18,BK90,East Williamsburg,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.891267
3,09/05/2015,204337,217969,POINT (-73.93445615919741 40.713537494833226),10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,53,18,BK90,East Williamsburg,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.253136
4,08/30/2015,189565,223043,POINT (-73.97597938483258 40.66677775537875),21,0,OnCurb,Alive,Good,Tilia americana,...,44,21,BK37,Park Slope-Gowanus,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.425999


In [4]:
trees_initial.shape

(683788, 42)

In [5]:
trees_initial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683788 entries, 0 to 683787
Data columns (total 42 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   created_at  683788 non-null  object 
 1   tree_id     683788 non-null  int64  
 2   block_id    683788 non-null  int64  
 3   the_geom    683788 non-null  object 
 4   tree_dbh    683788 non-null  int64  
 5   stump_diam  683788 non-null  int64  
 6   curb_loc    683788 non-null  object 
 7   status      683788 non-null  object 
 8   health      652172 non-null  object 
 9   spc_latin   652169 non-null  object 
 10  spc_common  652169 non-null  object 
 11  steward     652173 non-null  object 
 12  guards      652172 non-null  object 
 13  sidewalk    652172 non-null  object 
 14  user_type   683788 non-null  object 
 15  problems    652124 non-null  object 
 16  root_stone  683788 non-null  object 
 17  root_grate  683788 non-null  object 
 18  root_other  683788 non-null  object 
 19  tr

In [6]:
trees_initial.describe()

,tree_id,block_id,tree_dbh,stump_diam,zipcode,cb_num,borocode,cncldist,st_assem,st_senate,boro_ct,Latitude,longitude,x_sp,y_sp
count,683788.000000,683788.000000,683788.000000,683788.000000,683788.000000,683788.000000,683788.000000,683788.000000,683788.000000,683788.000000,6.837880e+05,683788.000000,683788.000000,6.837880e+05,683788.000000
mean,365205.011085,313793.096236,11.279787,0.432463,10916.246044,343.505404,3.358500,29.943181,50.791583,20.615781,3.404914e+06,40.701261,-73.924060,1.005280e+06,194798.424624
std,208122.092902,114839.024312,8.723042,3.290241,651.553364,115.740601,1.166746,14.328531,18.966520,7.390844,1.175863e+06,0.090311,0.123583,3.428505e+04,32902.061114
min,3.000000,100002.000000,0.000000,0.000000,83.000000,101.000000,1.000000,1.000000,23.000000,10.000000,1.000201e+06,40.498466,-74.254965,9.133493e+05,120973.792223
25%,186582.750000,221556.000000,4.000000,0.000000,10451.000000,302.000000,3.000000,19.000000,33.000000,14.000000,3.011700e+06,40.631928,-73.980500,9.896578e+05,169515.153719
50%,366214.500000,319967.000000,9.000000,0.000000,11214.000000,402.000000,4.000000,30.000000,52.000000,21.000000,4.008100e+06,40.700612,-73.912911,1.008386e+06,194560.252497
75%,546170.250000,404624.000000,16.000000,0.000000,11365.000000,412.000000,4.000000,43.000000,64.000000,25.000000,4.103202e+06,40.762228,-73.834910,1.029991e+06,217019.571916
max,722694.000000,999999.000000,450.000000,140.000000,11697.000000,503.000000,5.000000,51.000000,87.000000,36.000000,5.032300e+06,40.912918,-73.700488,1.067248e+06,271894.092088


* Rename some columns.

In [7]:
trees_initial.columns = ['created_at', 'tree_id', 'block_id', 'the_geom', 'tree_dbh',
       'stump_diam', 'curb_loc', 'status', 'health', 'spc_latin', 'spc_common',
       'steward', 'guards', 'sidewalk', 'user_type', 'problems', 'root_stone',
       'root_grate', 'root_other', 'trunk_wire', 'trunk_light', 'trunk_other',
       'branch_light', 'branch_shoe', 'branch_other', 'address', 'zipcode',
       'zip_city', 'cb_num', 'borocode', 'boroname', 'council_dist', 'st_assem',
       'st_senate', 'nta', 'nta_name', 'boro_ct', 'state', 'latitude',
       'longitude', 'x_sp', 'y_sp']

* Remove stumps and dead trees.

In [8]:
trees_initial.status.value_counts()

Alive    652173
Stump     17654
Dead      13961
Name: status, dtype: int64

In [9]:
# Get indices of dead trees and stumps
dead = trees_initial[trees_initial.status.isin(['Dead', 'Stump'])].index
 
# Delete these row indices from dataFrame
trees_initial.drop(dead, inplace=True)

trees_initial.shape

(652173, 42)

* Drop some unneccesary columns and set the index to the **tree_id**.

In [10]:
drop_cols = ['created_at', 'the_geom', 'stump_diam', 'status', 'spc_latin', 'problems', 'address', 'zipcode',
             'zip_city', 'borocode', 'boro_ct', 'state', 'x_sp', 'y_sp']
trees_initial.drop(columns=drop_cols, inplace=True)
trees_initial.set_index(['tree_id'], inplace=True)
trees_initial.shape

(652173, 27)

* Save a copy.

In [11]:
# uncomment to save
# trees_initial.to_csv('data/nyc_trees_2015_initial_clean.csv')

In [12]:
trees_mapmaking = pd.read_csv('data/nyc_trees_2015_initial_clean.csv', index_col=0)
trees_mapmaking.head()

,block_id,tree_dbh,curb_loc,health,spc_common,steward,guards,sidewalk,user_type,root_stone,...,branch_other,cb_num,boroname,council_dist,st_assem,st_senate,nta,nta_name,latitude,longitude
tree_id,,,,,,,,,,,,,,,,,,,,,
180683,348711,3,OnCurb,Fair,red maple,None,None,NoDamage,TreesCount Staff,No,...,No,406,Queens,29,28,16,QN17,Forest Hills,40.723092,-73.844215
200540,315986,21,OnCurb,Fair,pin oak,None,None,Damage,TreesCount Staff,Yes,...,No,407,Queens,19,27,11,QN49,Whitestone,40.794111,-73.818679
204026,218365,3,OnCurb,Good,honeylocust,1or2,None,Damage,Volunteer,No,...,No,301,Brooklyn,34,50,18,BK90,East Williamsburg,40.717581,-73.936608
204337,217969,10,OnCurb,Good,honeylocust,None,None,Damage,Volunteer,Yes,...,No,301,Brooklyn,34,53,18,BK90,East Williamsburg,40.713537,-73.934456
189565,223043,21,OnCurb,Good,American linden,None,None,Damage,Volunteer,Yes,...,No,306,Brooklyn,39,44,21,BK37,Park Slope-Gowanus,40.666778,-73.975979


## Mapmaking

In [48]:
# Make a list of latitude, longitude, and health status for all datapoints
latlon = [(lat, lon, health) for lat, lon, health in zip(list(trees_mapmaking.latitude),
                                                         list(trees_mapmaking.longitude),
                                                         list(trees_mapmaking.health))]
latlon[:5]

[(40.72309177, -73.84421522, 'Fair'),
 (40.79411067, -73.81867946, 'Fair'),
 (40.71758074, -73.93660770000002, 'Good'),
 (40.71353749, -73.93445616, 'Good'),
 (40.66677776, -73.97597938, 'Good')]

* Uncomment code below to make a map of all the datapoints. *WARNING: takes awhile and the output html file is over 300MB and very laggy.*

In [49]:
# tree_map = folium.Map(location=[40.700991, -73.924587], zoom_start=11)

# for coord in latlon:
#     if coord[2] == 'Good':
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='green').add_to(tree_map)
#     elif coord[2] == 'Fair':
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='yellow').add_to(tree_map)
#     else:
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='red').add_to(tree_map)

# tree_map.save('maps/tree_health_map.html')

* Let's make some maps on a smaller scale.
    * First let's look at the five community board areas with the highest proportion of trees in 'Good' health.
    * Then let's look at the five community board areas with the highest proportion of trees in 'Poor' health.
    
### Healthy trees map

In [43]:
healthiest = trees_mapmaking.groupby(['nta']).health.value_counts(normalize=True).unstack().sort_values('Good', ascending=False).head(10)
healthiest.index


Index(['MN50', 'QN02', 'QN49', 'MN13', 'BK93', 'SI07', 'SI28', 'BX37', 'SI35',
       'QN07'],
      dtype='object', name='nta')

In [44]:
# create new dataframe with just these ten neighborhoods
most_good_trees = trees_mapmaking[trees_mapmaking.nta.isin(list(healthiest.index))]
# find the average coordinates, so we can set the center of our map
most_good_trees[['latitude', 'longitude']].mean()

latitude     40.708752
longitude   -73.945031
dtype: float64

In [45]:
# Make a list of latitude, longitude, and health status for all datapoints
latlon_best = [(lat, lon, health) for lat, lon, health in zip(list(most_good_trees.latitude),
                                                         list(most_good_trees.longitude),
                                                         list(most_good_trees.health))]
latlon_best[:5]

[(40.79411067, -73.81867946, 'Fair'),
 (40.7337165, -73.97705764, 'Fair'),
 (40.793138, -73.81946649, 'Good'),
 (40.73357762, -73.97672526, 'Fair'),
 (40.73346807, -73.97646308, 'Good')]

In [54]:
healthy_nta_map = folium.Map(location=[40.708752, -73.945031], zoom_start=11)

for coord in latlon_best:
    if coord[2] == 'Good':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='green').add_to(healthy_nta_map)
    elif coord[2] == 'Fair':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='yellow').add_to(healthy_nta_map)
    else:
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='red').add_to(healthy_nta_map)

healthy_nta_map.save('maps/healthy_nta_map.html')

### Poorest health trees map

In [67]:
poorest_health = trees_mapmaking.groupby(['nta']).health.value_counts(normalize=True).unstack().sort_values('Poor', ascending=False).head(10)
poorest_health.index

Index(['QN10', 'QN12', 'MN31', 'MN32', 'MN36', 'QN20', 'MN20', 'MN17', 'MN01',
       'MN34'],
      dtype='object', name='nta')

In [50]:
poorest_health = trees_mapmaking.groupby(['nta']).health.value_counts(normalize=True).unstack().sort_values('Good', ascending=True).head(10)
poorest_health.index


Index(['QN12', 'QN10', 'MN17', 'MN31', 'BK23', 'BK21', 'MN20', 'MN35', 'MN01',
       'BX01'],
      dtype='object', name='nta')

In [51]:
# create new dataframe with just these five community boards
most_poor_trees = trees_mapmaking[trees_mapmaking.nta.isin(list(poorest_health.index))]
# find the average coordinates, so we can set the center of our map
most_poor_trees[['latitude', 'longitude']].mean()

latitude     40.710840
longitude   -73.904969
dtype: float64

In [52]:
# Make a list of latitude, longitude, and health status for all datapoints
latlon_worst = [(lat, lon, health) for lat, lon, health in zip(list(most_poor_trees.latitude),
                                                         list(most_poor_trees.longitude),
                                                         list(most_poor_trees.health))]
latlon_worst[:5]

[(40.74503399, -73.98253015, 'Fair'),
 (40.74829709, -73.98065645, 'Good'),
 (40.59626688, -73.77234286, 'Fair'),
 (40.59683648, -73.77245394, 'Poor'),
 (40.77277225, -73.95532709999998, 'Good')]

In [55]:
poor_health_nta_map = folium.Map(location=[40.710840, -73.904969], zoom_start=11)

for coord in latlon_worst:
    if coord[2] == 'Good':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='green').add_to(poor_health_nta_map)
    elif coord[2] == 'Fair':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='yellow').add_to(poor_health_nta_map)
    else:
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='red').add_to(poor_health_nta_map)

poor_health_nta_map.save('maps/poor_health_nta_map.html')

### Bushwick trees
* Make a map on a smaller scale, using my neighborhood of Bushwick as an example. *NOTE: you can use any community board number or combination of neighborhoods, for example.*

In [57]:
bushwick = trees_mapmaking[trees_mapmaking.cb_num == 304]
# find the average coordinates, so we can set the center of our map
bushwick[['latitude', 'longitude']].mean()

latitude     40.696207
longitude   -73.918556
dtype: float64

In [58]:
latlon_bushwick = [(lat, lon, health) for lat, lon, health in zip(list(bushwick.latitude),
                                                                  list(bushwick.longitude),
                                                                  list(bushwick.health))]
latlon_bushwick[:5]

[(40.69775112, -73.90933909, 'Fair'),
 (40.70233782, -73.91521777, 'Good'),
 (40.69791922, -73.90830664, 'Good'),
 (40.68280728, -73.90979496, 'Good'),
 (40.70256662, -73.91498455, 'Good')]

In [59]:
bushwick_map = folium.Map(location=[40.696207, -73.918556], zoom_start=15)

for coord in latlon_bushwick:
    if coord[2] == 'Good':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='green').add_to(bushwick_map)
    elif coord[2] == 'Fair':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='yellow').add_to(bushwick_map)
    else:
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='red').add_to(bushwick_map)

bushwick_map.save('maps/bushwick_tree_health_map.html')

### Maps by census taker (professional vs. volunteer)

In [63]:
# create new dataframe with just professional census takers
pro_trees = trees_mapmaking[trees_mapmaking.user_type.isin(['TreesCount Staff', 'NYC Parks Staff'])]
# create new dataframe with just volunteer census takers
vol_trees = trees_mapmaking[trees_mapmaking.user_type == 'Volunteer']

In [64]:
# Make a list of latitude, longitude, and health status for all datapoints
latlon_pros = [(lat, lon, health) for lat, lon, health in zip(list(pro_trees.latitude),
                                                         list(pro_trees.longitude),
                                                         list(pro_trees.health))]

latlon_vols = [(lat, lon, health) for lat, lon, health in zip(list(vol_trees.latitude),
                                                         list(vol_trees.longitude),
                                                         list(vol_trees.health))]

* Uncomment code below to make a map of all the datapoints. *WARNING: takes awhile and the output html file is over 300MB and very laggy.*

In [65]:
# pro_map = folium.Map(location=[40.700991, -73.924587], zoom_start=11)

# for coord in latlon_pros:
#     if coord[2] == 'Good':
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='green').add_to(pro_map)
#     elif coord[2] == 'Fair':
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='yellow').add_to(pro_map)
#     else:
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='red').add_to(pro_map)

# pro_map.save('maps/pro_census_map.html')

In [66]:
# vol_map = folium.Map(location=[40.700991, -73.924587], zoom_start=11)

# for coord in latlon_vols:
#     if coord[2] == 'Good':
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='green').add_to(vol_map)
#     elif coord[2] == 'Fair':
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='yellow').add_to(vol_map)
#     else:
#         folium.Circle(location=[coord[0], coord[1]], radius=1, color='red').add_to(vol_map)

# vol_map.save('maps/vol_census_map.html')

### Best/worst community board

In [78]:
trees_mapmaking.groupby(['cb_num']).health.value_counts(normalize=True).unstack().sort_values('Good', ascending=False).head(1)

health,Fair,Good,Poor
cb_num,,,
210,0.095958,0.877063,0.026979


In [77]:
trees_mapmaking.groupby(['cb_num']).health.value_counts(normalize=True).unstack().sort_values('Poor', ascending=False).head(1)


health,Fair,Good,Poor
cb_num,,,
414,0.254523,0.64509,0.100386


In [80]:
best_cb = trees_mapmaking[trees_mapmaking.cb_num == 210]
# find the average coordinates, so we can set the center of our map
best_cb[['latitude', 'longitude']].mean()

latitude     40.839529
longitude   -73.823598
dtype: float64

In [79]:
worst_cb = trees_mapmaking[trees_mapmaking.cb_num == 414]
# find the average coordinates, so we can set the center of our map
worst_cb[['latitude', 'longitude']].mean()

latitude     40.593158
longitude   -73.794398
dtype: float64

In [81]:
# Make a list of latitude, longitude, and health status for all datapoints
latlon_best_cb = [(lat, lon, health) for lat, lon, health in zip(list(best_cb.latitude),
                                                         list(best_cb.longitude),
                                                         list(best_cb.health))]

latlon_worst_cb = [(lat, lon, health) for lat, lon, health in zip(list(worst_cb.latitude),
                                                         list(worst_cb.longitude),
                                                         list(worst_cb.health))]

In [82]:
best_cb_map = folium.Map(location=[40.839529, -73.823598], zoom_start=15)

for coord in latlon_best_cb:
    if coord[2] == 'Good':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='green').add_to(best_cb_map)
    elif coord[2] == 'Fair':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='yellow').add_to(best_cb_map)
    else:
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='red').add_to(best_cb_map)

best_cb_map.save('maps/best_cb_map.html')

In [83]:
worst_cb_map = folium.Map(location=[40.593158, -73.794398], zoom_start=15)

for coord in latlon_worst_cb:
    if coord[2] == 'Good':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='green').add_to(worst_cb_map)
    elif coord[2] == 'Fair':
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='yellow').add_to(worst_cb_map)
    else:
        folium.Circle(location=[coord[0], coord[1]], radius=1, color='red').add_to(worst_cb_map)

worst_cb_map.save('maps/worst_cb_map.html')